In [1]:
import json
import os
from typing import List, Dict, Tuple, Optional, Callable
import re
from pathlib import Path
import PyPDF2
from langchain_ollama import OllamaLLM

# 간단한 progress reporter (콘솔 출력용)
def simple_progress_reporter(stage: str, status: str, details: Optional[Dict] = None):
    from datetime import datetime
    ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    details = details or {}
    print(f"[{ts}] {stage.upper():8} | {status.upper():7} | {details}")


In [2]:
# ============================================================================
# 1. 문서 로더 및 전처리
# ============================================================================

class DocumentProcessor:
    """다양한 형식의 문서를 읽고 청킹하는 클래스"""
    
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap # 청크 1000자, 겹침 200자
    
    def load_documents(self, file_paths: List[str]) -> List[Dict[str, str]]:
        """여러 형식의 문서 로드"""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.txt'):
                content = self._load_txt(file_path)
            elif file_path.endswith('.md'):
                content = self._load_markdown(file_path)
            elif file_path.endswith('.pdf'):
                content = self._load_pdf(file_path)
            elif file_path.endswith('.json'):
                content = self._load_json(file_path)
            else:
                continue
            
            if content:
                documents.append({
                    'source': file_path,
                    'content': content,
                    'type': file_path.split('.')[-1] #txt, md, pdf, json 등
                })
        
        return documents
    
    def _load_txt(self, file_path: str) -> str:
        """텍스트 파일 로드"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return ""
    
    def _load_markdown(self, file_path: str) -> str:
        """마크다운 파일 로드"""
        return self._load_txt(file_path)
    
    def _load_pdf(self, file_path: str) -> str:
        """PDF 파일 로드 (PyPDF2 필요)"""
        try:
            import PyPDF2
            with open(file_path, 'rb') as f:
                pdf_reader = PyPDF2.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text #text=전체 PDF 텍스트
        except Exception as e:
            print(f"Error loading PDF {file_path}: {e}")
            return ""
    
    def _load_json(self, file_path: str) -> str:
        """JSON 파일 로드 (텍스트 추출)"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # JSON 데이터를 문자열로 변환
                if isinstance(data, list):
                    return "\n".join(str(item) for item in data)
                elif isinstance(data, dict):
                    return json.dumps(data, ensure_ascii=False, indent=2)
                else:
                    return str(data)
        except Exception as e:
            print(f"Error loading JSON {file_path}: {e}")
            return ""
    
    def chunk_text(self, text: str) -> List[str]:
        """텍스트를 청크로 분할"""
        chunks = []
        step = self.chunk_size - self.chunk_overlap
        
        for i in range(0, len(text), step):
            chunk = text[i:i + self.chunk_size]
            if len(chunk) > 100:  # 너무 짧은 청크 제외
                chunks.append(chunk)
        
        return chunks
    
    def preprocess_documents(self, documents: List[Dict]) -> List[str]:
        """문서 전처리 및 청킹"""
        all_chunks = []
        
        for doc in documents:
            # 불필요한 공백 제거
            content = re.sub(r'\s+', ' ', doc['content'])
            # 청킹
            chunks = self.chunk_text(content)
            all_chunks.extend(chunks)
        
        print(f"총 {len(all_chunks)}개 청크 생성")
        return all_chunks#

In [3]:
# ============================================================================
# 2. LLM 기반 자동 생성 (Ollama 또는 API 사용)
# ============================================================================

from tqdm.auto import tqdm

class CaptionDataGeneratorKor:
    """LLM을 사용하여 caption-interpretation 쌍 자동 생성"""
    
    def __init__(self, model_name: str = "mistral", use_local: bool = True):
        """
        Parameters:
        - model_name: 사용할 LLM 모델명
        - use_local: Ollama 같은 로컬 모델 사용 여부
        """
        self.model_name = model_name
        self.use_local = use_local
        self.llm = None
        self._init_llm()
    
    def _init_llm(self):
        """LLM 초기화"""
        if self.use_local:
            try:
                self.llm = OllamaLLM(
                    model=self.model_name,
                    temperature=0.7,
                    num_ctx=2048,
                )
                print(f"✓ Ollama 모델 로드: {self.model_name}")
            except Exception as e:
                print(f"✗ Ollama 로드 실패: {e}")
                print("  대안: OpenAI API 사용")
        else:
            try:
                from langchain.llms import OpenAI
                self.llm = OpenAI(
                    temperature=0.7,
                    max_tokens=500
                )
                print("✓ OpenAI 모델 로드")
            except Exception as e:
                print(f"✗ OpenAI 로드 실패: {e}")
    
    def generate_qa_pairs(self, chunk: str, num_pairs: int = 2) -> List[Dict[str, str]]:
        """문서 청크로부터 QA 쌍 생성"""
        
        if not self.llm:
            print("✗ LLM이 초기화되지 않았습니다")
            return []
        
        prompt = f"""다음 문서 내용을 읽고, HTP 검사 해석과 관련하여 최대{num_pairs}개의 캡션-해석 쌍을 생성해주세요.

문서 내용:
{chunk}

요구사항:
1. 각 쌍은 실제 HTP 검사에서 나올 수 있는 HTP 그림 관련 캡션과 심리학적 해석
2. JSON 배열 형식으로 반환
3. 형식: {{"caption": "캡션", "interpretation": "해석"}}
4. 캡션은 간결하게, 해석은 충분히 상세하게
5. 총 {num_pairs}개
6. 문서를 참고해서 제작이 불가능할 경우 빈 배열 반환
예시) {{"caption": "집의 크기가 크다", "interpretation": "대상자가 상상력이 풍부하거나 책임감이 강하다고 해석할 수 있다."}}

JSON 배열만 반환 (설명 없음):"""
        
        try:
            # 안전하게 중괄호를 이스케이프하여 llm wrapper 내부에서 .format이 사용되더라도 에러가 나지 않게 함
            safe_prompt = prompt.replace('{', '{{').replace('}', '}}')
            response = self.llm.invoke(safe_prompt)
            # JSON 파싱
            pairs = self._parse_json_response(response)
            return pairs
        except Exception as e:
            print(f"✗ 생성 중 오류: {e}")
            return []
    
    def _parse_json_response(self, response: str) -> List[Dict]:
        """응답에서 JSON 추출"""
        try:
            # 응답에서 JSON 배열 찾기
            json_start = response.find('[')
            json_end = response.rfind(']') + 1
            
            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                return json.loads(json_str)
        except json.JSONDecodeError:
            print("✗ JSON 파싱 실패")
            print(response)
            return []
        
        return []
    
    def batch_generate(self, chunks: List[str], num_pairs_per_chunk: int = 2) -> List[Dict]:
        """여러 청크로부터 일괄 생성"""
        all_pairs = []
        
        print(f"\n[자동 생성 시작] 총 {len(chunks)}개 청크 처리 중...")
        
        # tqdm으로 진행바 표시
        for chunk in tqdm(chunks, desc='Generating', unit='chunk'):
            pairs = self.generate_qa_pairs(chunk, num_pairs_per_chunk)
            all_pairs.extend(pairs)
        
        print(f"✓ 총 {len(all_pairs)}개 쌍 생성 완료\n")
        return all_pairs


In [4]:
# ============================================================================
# 2. LLM 기반 자동 생성 (Ollama 또는 API 사용)
# ============================================================================

from tqdm.auto import tqdm

class CaptionDataGeneratorEng:
    """LLM을 사용하여 caption-interpretation 쌍 자동 생성"""
    
    def __init__(self, model_name: str = "mistral", use_local: bool = True):
        """
        Parameters:
        - model_name: 사용할 LLM 모델명
        - use_local: Ollama 같은 로컬 모델 사용 여부
        """
        self.model_name = model_name
        self.use_local = use_local
        self.llm = None
        self._init_llm()
    
    def _init_llm(self):
        """LLM 초기화"""
        if self.use_local:
            try:
                self.llm = OllamaLLM(
                    model=self.model_name,
                    temperature=0.7,
                    num_ctx=2048,
                )
                print(f"✓ Ollama 모델 로드: {self.model_name}")
            except Exception as e:
                print(f"✗ Ollama 로드 실패: {e}")
                print("  대안: OpenAI API 사용")
        else:
            try:
                from langchain.llms import OpenAI
                self.llm = OpenAI(
                    temperature=0.7,
                    max_tokens=500
                )
                print("✓ OpenAI 모델 로드")
            except Exception as e:
                print(f"✗ OpenAI 로드 실패: {e}")
    
    def generate_qa_pairs(self, chunk: str, num_pairs: int = 2) -> List[Dict[str, str]]:
        """문서 청크로부터 QA 쌍 생성"""
        
        if not self.llm:
            print("✗ LLM이 초기화되지 않았습니다")
            return []
        
        prompt = f"""Read the following document content and generate up to {num_pairs} caption-interpretation pairs related to HTP (House-Tree-Person) test interpretation.


Document Content:
{chunk}


Requirements:
1. Each pair consists of an HTP drawing-related caption and psychological interpretation that could occur in an actual HTP assessment
2. Return in JSON array format
3. Format: {{"caption": "caption text", "interpretation": "interpretation text"}}
4. Keep captions concise; provide detailed interpretations
5. Total of {num_pairs} pairs
6. If it's impossible to generate pairs based on the document, return an empty array
Example) {{"caption": "The house size is large", "interpretation": "The subject may be interpreted as having a rich imagination or strong sense of responsibility."}}

Return only JSON array (no additional explanation):"""
        
        try:
            # 안전하게 중괄호 이스케이프
            safe_prompt = prompt.replace('{', '{{').replace('}', '}}')
            response = self.llm.invoke(safe_prompt)
            # JSON 파싱
            pairs = self._parse_json_response(response)
            return pairs
        except Exception as e:
            print(f"✗ 생성 중 오류: {e}")
            return []
    
    def _parse_json_response(self, response: str) -> List[Dict]:
        """응답에서 JSON 추출"""
        try:
            # 응답에서 JSON 배열 찾기
            json_start = response.find('[')
            json_end = response.rfind(']') + 1
            
            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                return json.loads(json_str)
        except json.JSONDecodeError:
            print("✗ JSON 파싱 실패")
            print(response)
            return []
        
        return []
    
    def batch_generate(self, chunks: List[str], num_pairs_per_chunk: int = 2) -> List[Dict]:
        """여러 청크로부터 일괄 생성"""
        all_pairs = []
        
        print(f"\n[자동 생성 시작] 총 {len(chunks)}개 청크 처리 중...")
        
        # tqdm으로 진행바 표시
        for chunk in tqdm(chunks, desc='Generating', unit='chunk'):
            pairs = self.generate_qa_pairs(chunk, num_pairs_per_chunk)
            all_pairs.extend(pairs)
        
        print(f"✓ 총 {len(all_pairs)}개 쌍 생성 완료\n")
        return all_pairs


In [5]:
# ============================================================================
# 3. 품질 검증 및 중복 제거
# ============================================================================

class DataQualityChecker:
    """생성된 데이터의 품질 검증"""
    
    @staticmethod
    def validate_pair(pair: Dict) -> bool:
        """단일 쌍 검증"""
        if not isinstance(pair, dict):
            return False
        
        if 'caption' not in pair or 'interpretation' not in pair:
            return False
        
        caption = str(pair['caption']).strip()
        interpretation = str(pair['interpretation']).strip()
        
        # 최소 길이 확인 캡션과 해석의 길이가 기준 미만인 경우 탈락
        if len(caption) < 5 or len(interpretation) < 20:
            return False
        
        # 반복 문자 확인
        if DataQualityChecker._has_repetition(caption) or DataQualityChecker._has_repetition(interpretation):
            return False
        
        return True
    
    @staticmethod
    def _has_repetition(text: str, threshold: float = 0.5) -> bool:
        """반복된 문자 검사"""
        words = text.split()
        if len(words) < 3:
            return False
        
        unique_ratio = len(set(words)) / len(words)#문장에 한번만 나타나는 단어의 비율이 낮은 경우 탈락
        return unique_ratio < threshold
    
    @staticmethod
    def remove_duplicates(pairs: List[Dict]) -> List[Dict]:
        """중복 제거 (캡션 기준)"""
        seen = set()
        unique_pairs = []
        
        for pair in pairs:
            caption = pair.get('caption', '').lower().strip()
            if caption not in seen and DataQualityChecker.validate_pair(pair):
                seen.add(caption)
                unique_pairs.append(pair)#캡션이 동일한데 해석이 다른 경우 탈락 시킴
        
        return unique_pairs
    
    @staticmethod
    def filter_pairs(pairs: List[Dict]) -> Tuple[List[Dict], int]:
        """품질 검증 및 필터링"""
        valid_pairs = [p for p in pairs if DataQualityChecker.validate_pair(p)]
        invalid_count = len(pairs) - len(valid_pairs)
        
        if invalid_count > 0:
            print(f"⚠️ {invalid_count}개 쌍 제외 (품질 미흡)")
        
        return valid_pairs, invalid_count

In [6]:
# ============================================================================
# 4. 데이터셋 저장
# ============================================================================

class DatasetSaver:
    """생성된 데이터셋을 다양한 형식으로 저장"""
    
    @staticmethod
    def save_csv(pairs: List[Dict], filename: str = "htp_training_data.csv"):
        """CSV 형식으로 저장"""
        import csv
        
        with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['C_ID', 'T_ID', 'Text', 'Completion'])
            writer.writeheader()
            
            for c_id, pair in enumerate(pairs):
                writer.writerow({
                    'C_ID': c_id,
                    'T_ID': 0,
                    'Text': f"HTP 검사 이미지 캡션: {pair['caption']}",
                    'Completion': pair['interpretation']
                })
        
        print(f"✓ CSV 저장: {filename}")
        return filename
    
    @staticmethod
    def save_jsonl(pairs: List[Dict], filename: str = "htp_training_data.jsonl"):
        """JSONL 형식으로 저장"""
        with open(filename, 'w', encoding='utf-8') as f:
            for c_id, pair in enumerate(pairs):
                data = {
                    'C_ID': c_id,
                    'T_ID': 0,
                    'Text': f"HTP 검사 이미지 캡션: {pair['caption']}",
                    'Completion': pair['interpretation']
                }
                f.write(json.dumps(data, ensure_ascii=False) + '\n')
        
        print(f"✓ JSONL 저장: {filename}")
        return filename
    
    @staticmethod
    def save_alpaca_format(pairs: List[Dict], filename: str = "htp_alpaca_format.json"):
        """Alpaca 형식으로 저장"""
        alpaca_data = []
        
        for pair in pairs:
            alpaca_data.append({
                "instruction": "다음 HTP 검사 이미지 캡션을 심리학적으로 해석해주세요",
                "input": pair['caption'],
                "output": pair['interpretation']
            })
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(alpaca_data, f, ensure_ascii=False, indent=2)
        
        print(f"✓ Alpaca 형식 저장: {filename}")
        return filename
    
    @staticmethod
    def save_raw_json(pairs: List[Dict], filename: str = "htp_raw_pairs.json"):
        """원본 쌍을 JSON으로 저장"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(pairs, f, ensure_ascii=False, indent=2)
        
        print(f"✓ 원본 쌍 저장: {filename}")
        return filename

In [7]:
# ============================================================================
# 5. 전체 파이프라인
# ============================================================================

class FullPipeline:
    """문서 입력부터 데이터셋 생성까지 완전한 파이프라인"""
    
    def __init__(self, model_name: str = "mistral", use_local: bool = True,language='kor'):
        self.doc_processor = DocumentProcessor()
        if language=='kor':
            self.data_generator = CaptionDataGeneratorKor(model_name, use_local)
        else:
            self.data_generator=CaptionDataGeneratorEng(model_name,use_local)
        self.quality_checker = DataQualityChecker()
        self.saver = DatasetSaver()
    
    def run(self, 
            file_paths: List[str],
            num_pairs_per_chunk: int = 2,
            output_dir: str = "./output",
            progress_callback: Optional[Callable[[str,str,Optional[Dict]], None]] = None) -> Dict:
        """
        전체 파이프라인 실행
        
        Parameters:
        - file_paths: 입력 문서 경로 리스트
        - num_pairs_per_chunk: 청크당 생성할 쌍의 수
        - output_dir: 출력 디렉토리
        - progress_callback: (stage, status, details) 형식의 콜백 함수
        
        Returns:
        - 결과 정보 딕셔너리
        """
        
        def _report(stage: str, status: str, details: Optional[Dict] = None):
            if progress_callback:
                try:
                    progress_callback(stage, status, details)
                except Exception as e:
                    print(f"[WARN] progress_callback raised an exception: {e}")

        # 디렉토리 생성
        Path(output_dir).mkdir(exist_ok=True)
        
        print("=" * 80)
        print("HTP 캡션-해석 쌍 자동 생성 파이프라인")
        print("=" * 80 + "\n")
        
        # 1단계: 문서 로드
        _report('load', 'start', {'file_paths': file_paths})
        print("[1단계] 문서 로드")
        print("-" * 80)
        documents = self.doc_processor.load_documents(file_paths)
        print(f"✓ {len(documents)}개 문서 로드됨\n")
        if not documents:
            _report('load', 'failure', {'file_paths': file_paths})
            raise RuntimeError(f"[오류] 문서 로드 실패: 제공된 경로에서 문서를 불러오지 못했습니다. file_paths={file_paths}")
        _report('load', 'success', {'documents': len(documents)})
        
        # 2단계: 문서 청킹
        _report('chunk', 'start', {'documents': len(documents)})
        print("[2단계] 문서 청킹")
        print("-" * 80)
        chunks = self.doc_processor.preprocess_documents(documents)
        print(f"✓ 청킹 완료\n")
        if not chunks:
            _report('chunk', 'failure', {'documents': len(documents)})
            raise RuntimeError("[오류] 청킹 실패: 문서로부터 유의미한 청크를 생성하지 못했습니다. 문서 내용 혹은 전처리 설정(chunk_size/chunk_overlap)을 확인하세요.")
        _report('chunk', 'success', {'chunks': len(chunks)})
        
        # 3단계: 데이터 생성
        _report('generate', 'start', {'chunks': len(chunks)})
        print("[3단계] LLM 기반 데이터 생성")
        print("-" * 80)
        raw_pairs = self.data_generator.batch_generate(chunks, num_pairs_per_chunk)
        if not raw_pairs:
            _report('generate', 'failure', {'chunks': len(chunks)})
            raise RuntimeError("[오류] 데이터 생성 실패: LLM이 유효한 캡션-해석 쌍을 생성하지 못했습니다. LLM 설정 및 입력 청크를 확인하세요.")
        _report('generate', 'success', {'raw_pairs': len(raw_pairs)})
        
        # 4단계: 품질 검증
        _report('validate', 'start', {'raw_pairs': len(raw_pairs)})
        print("[4단계] 품질 검증")
        print("-" * 80)
        valid_pairs, invalid_count = self.quality_checker.filter_pairs(raw_pairs)
        print(f"✓ 유효한 쌍: {len(valid_pairs)}개\n")
        if len(valid_pairs) == 0:
            _report('validate', 'failure', {'raw_pairs': len(raw_pairs), 'invalid_count': invalid_count})
            raise RuntimeError("[오류] 품질 검증 결과 유효한 쌍이 없습니다. 필터 기준(DataQualityChecker)을 완화하거나 LLM 출력을 확인하세요.")
        _report('validate', 'success', {'valid_pairs': len(valid_pairs), 'invalid_count': invalid_count})
        
        # 5단계: 중복 제거
        _report('dedupe', 'start', {'valid_pairs': len(valid_pairs)})
        print("[5단계] 중복 제거")
        print("-" * 80)
        unique_pairs = self.quality_checker.remove_duplicates(valid_pairs)
        duplicate_count = len(valid_pairs) - len(unique_pairs)
        if duplicate_count > 0:
            print(f"⚠️ {duplicate_count}개 중복 제거")
        print(f"✓ 최종 쌍: {len(unique_pairs)}개\n")
        if len(unique_pairs) == 0:
            _report('dedupe', 'failure', {'valid_pairs': len(valid_pairs)})
            raise RuntimeError("[오류] 중복 제거 후 유효한 쌍이 없습니다. 생성된 쌍을 확인하세요.")
        _report('dedupe', 'success', {'unique_pairs': len(unique_pairs), 'duplicate_count': duplicate_count})
        
        # 6단계: 데이터 저장
        _report('save', 'start', {'unique_pairs': len(unique_pairs), 'output_dir': output_dir})
        print("[6단계] 데이터 저장")
        print("-" * 80)
        try:
            csv_path = self.saver.save_csv(unique_pairs, f"{output_dir}/htp_training_data.csv")
            jsonl_path = self.saver.save_jsonl(unique_pairs, f"{output_dir}/htp_training_data.jsonl")
            alpaca_path = self.saver.save_alpaca_format(unique_pairs, f"{output_dir}/htp_alpaca_format.json")
            raw_path = self.saver.save_raw_json(unique_pairs, f"{output_dir}/htp_raw_pairs.json")
        except Exception as e:
            _report('save', 'failure', {'error': str(e)})
            raise RuntimeError(f"[오류] 데이터 저장 실패: {e}")

        # 저장 확인
        missing = []
        for p in (csv_path, jsonl_path, alpaca_path, raw_path):
            try:
                if not Path(p).exists():
                    missing.append(p)
            except Exception:
                missing.append(p)
        if missing:
            _report('save', 'failure', {'missing': missing})
            raise RuntimeError(f"[오류] 데이터 저장은 완료되었으나 다음 파일을 찾을 수 없습니다: {missing}")
        _report('save', 'success', {'paths': [csv_path, jsonl_path, alpaca_path, raw_path]})

        print("\n" + "=" * 80)
        print("✓ 완료!")
        print("=" * 80)
        _report('complete', 'success', {'unique_pairs': len(unique_pairs)})
        
        return {
            'total_documents': len(documents),
            'total_chunks': len(chunks),
            'raw_pairs': len(raw_pairs),
            'valid_pairs': len(valid_pairs),
            'unique_pairs': len(unique_pairs),
            'output_dir': output_dir
        }


In [8]:
import os
# 사용할 문서 경로
folder_path = "HTP_DATA"
file_paths=[os.path.join(folder_path, f) for f in os.listdir(folder_path)]
    
file_paths

['HTP_DATA\\000002419228_20251111110826.pdf',
 'HTP_DATA\\200000215097_20251111110528.pdf',
 'HTP_DATA\\buck1948.pdf',
 'HTP_DATA\\finetunning_data.txt',
 'HTP_DATA\\fpsyt-13-1041770.pdf',
 'HTP_DATA\\HTP_Drawing_Test.txt',
 'HTP_DATA\\HTP_Test_Interpertation_kor.txt',
 'HTP_DATA\\ilide.info-the-h-t-p-technique-a-qualitative-and-quantitative-scoring-john-n-buck-journal-o-pr_090eb8a4e23407a52ca8c7d1ace3827a.pdf',
 'HTP_DATA\\KCI_FI003005286.pdf',
 'HTP_DATA\\KCI_FI003198543.pdf',
 'HTP_DATA\\munevveriremhatipoglu.pdf',
 'HTP_DATA\\sampe_interpretation_of_house_tree_person_test.txt',
 'HTP_DATA\\브런치 집_나무_사람 해석 수정본 (1).txt',
 'HTP_DATA\\표 형태의 논문 해석.json']

In [9]:
# 파이프라인 실행
pipeline = FullPipeline(model_name="gemma3:latest", use_local=True,language='kor')

results = pipeline.run(
    file_paths=file_paths,
    num_pairs_per_chunk=2,
    output_dir="./htp_output1_kor"
)

# 결과 출력
print("\n[최종 통계]")
for key, value in results.items():
    print(f"{key}: {value}")

✓ Ollama 모델 로드: gemma3:latest
HTP 캡션-해석 쌍 자동 생성 파이프라인

[1단계] 문서 로드
--------------------------------------------------------------------------------
Error loading HTP_DATA\finetunning_data.txt: 'utf-8' codec can't decode byte 0xc1 in position 14: invalid start byte
Error loading HTP_DATA\finetunning_data.txt: 'utf-8' codec can't decode byte 0xc1 in position 14: invalid start byte
✓ 13개 문서 로드됨

[2단계] 문서 청킹
--------------------------------------------------------------------------------
총 1354개 청크 생성
✓ 청킹 완료

[3단계] LLM 기반 데이터 생성
--------------------------------------------------------------------------------

[자동 생성 시작] 총 1354개 청크 처리 중...
✓ 13개 문서 로드됨

[2단계] 문서 청킹
--------------------------------------------------------------------------------
총 1354개 청크 생성
✓ 청킹 완료

[3단계] LLM 기반 데이터 생성
--------------------------------------------------------------------------------

[자동 생성 시작] 총 1354개 청크 처리 중...


Generating:   0%|          | 0/1354 [00:00<?, ?chunk/s]

✗ JSON 파싱 실패
```json
[
  {
    "caption": "HTP 검사 결과표의 Net Weighted Score 계산",
    "interpretation": "Net Weighted Score는 ‘good’ score와 ‘flaw’ score의 차이로 계산되며, 이는 대상자의 검사 결과가 ‘good’ score에 대한 ‘flaw’ score의 영향력을 얼마나 상쇄했는지를 나타냅니다.  Net Weighted Score의 값은 대상자의 검사 결과가 ‘good’ score에 대한 ‘flaw’ score의 영향을 얼마나 상쇄했는지를 보여주므로,  정량적인 비교를 위해 중요합니다."
  },
  {
    "caption": "Means Table 활용 및 층위별 평균 추정",
    "interpretation": "Means Table는 HTP 검사 결과를 다양한 지능 및 요인 수준별 평균 값과 비교하여 대상자의 총 raw score를 추정하는 데 사용됩니다.  "+" 기호는 평균보다 약간 높은 값을, "-" 기호는 평균보다 약간 낮은 값을 나타내므로,  각 지능 및 요인 수준에 대한 보다 정확한 평가를 제공합니다."
  }
]
```
✓ 총 2706개 쌍 생성 완료

[4단계] 품질 검증
--------------------------------------------------------------------------------
⚠️ 2개 쌍 제외 (품질 미흡)
✓ 유효한 쌍: 2704개

[5단계] 중복 제거
--------------------------------------------------------------------------------
⚠️ 23개 중복 제거
✓ 최종 쌍: 2681개

[6단계] 데이터 저장
--------------------------------------------------------------------------------
✓ CSV 저장: ./htp_output1_kor/htp_trainin

In [10]:
# 파이프라인 실행
pipeline = FullPipeline(model_name="gemma3:latest", use_local=True,language='eng')

results = pipeline.run(
    file_paths=file_paths,
    num_pairs_per_chunk=2,
    output_dir="./htp_output1_eng"
)

# 결과 출력
print("\n[최종 통계]")
for key, value in results.items():
    print(f"{key}: {value}")

✓ Ollama 모델 로드: gemma3:latest
HTP 캡션-해석 쌍 자동 생성 파이프라인

[1단계] 문서 로드
--------------------------------------------------------------------------------
Error loading HTP_DATA\finetunning_data.txt: 'utf-8' codec can't decode byte 0xc1 in position 14: invalid start byte
Error loading HTP_DATA\finetunning_data.txt: 'utf-8' codec can't decode byte 0xc1 in position 14: invalid start byte
✓ 13개 문서 로드됨

[2단계] 문서 청킹
--------------------------------------------------------------------------------
총 1354개 청크 생성
✓ 청킹 완료

[3단계] LLM 기반 데이터 생성
--------------------------------------------------------------------------------

[자동 생성 시작] 총 1354개 청크 처리 중...
✓ 13개 문서 로드됨

[2단계] 문서 청킹
--------------------------------------------------------------------------------
총 1354개 청크 생성
✓ 청킹 완료

[3단계] LLM 기반 데이터 생성
--------------------------------------------------------------------------------

[자동 생성 시작] 총 1354개 청크 처리 중...


Generating:   0%|          | 0/1354 [00:00<?, ?chunk/s]

✗ JSON 파싱 실패
```json
[
  {
    "caption": "The tree's branches are numerous and the house is expansive.",
    "interpretation": "This HTP suggests a potential for over-extroversion, a desire for numerous social connections, and possibly a feeling of being overwhelmed by possibilities, potentially indicating heightened anxiety or a need for control within social situations."
  },
  {
    "caption": "The house is small, and the tree’s branches are sparse.",
    "interpretation": "The interpretation suggests a diminished sense of self-efficacy, a feeling of vulnerability, and possibly a history of trauma or neglect, particularly in childhood. The lack of robust support (the sparse branches) coupled with the small house indicates a possible struggle with self-esteem and a heightened fear of external threats or abandonment.”}
]
```
✗ JSON 파싱 실패
```json
[
  {
    "caption": "The tree has many branches and leaves.",
    "interpretation": "The excessive number of branches and leaves in the tre

In [11]:
"""
HTP 데이터 처리 스크립트
- Instruction 영어 변환
- Category 자동 추출
- 데이터 검증 및 통계
- 여러 형식 저장

사용법:
    python htp_processor.py --input raw_data.json --output processed/
"""

import json
import re
import argparse
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from collections import Counter


class HTPDataProcessor:
    """
    HTP 데이터 처리 클래스
    - 대규모 데이터 처리 지원
    - 데이터 검증
    - 여러 출력 포맷 지원
    """
    
    def __init__(self, custom_categories: Optional[Dict[str, str]] = None):
        """
        초기화
        
        Args:
            custom_categories: 커스텀 카테고리 추가 가능
        """
        self.categories = {
            "tree": r"\btree\b",
            "person": r"\bperson\b",
            "house": r"\bhouse\b"
        }
        
        if custom_categories:
            self.categories.update(custom_categories)
        
        self.english_instruction = "Please provide a psychological interpretation of the following HTP test image caption"
    
    def extract_categories(self, text: str) -> str:
        """input 텍스트에서 category 추출"""
        text_lower = text.lower()
        found_categories = []
        
        for category, pattern in self.categories.items():
            if re.search(pattern, text_lower):
                found_categories.append(category)
        
        return ", ".join(found_categories) if found_categories else "other"
    
    def validate_item(self, item: Dict) -> Tuple[bool, str]:
        """
        데이터 아이템 유효성 검증
        
        Returns:
            (유효 여부, 에러 메시지)
        """
        required_fields = ["instruction", "input", "output"]
        
        for field in required_fields:
            if field not in item:
                return False, f"필수 필드 '{field}' 누락"
            if not isinstance(item[field], str) or not item[field].strip():
                return False, f"필드 '{field}'가 비어있음"
        
        return True, "OK"
    
    def process_single_item(self, item: Dict) -> Optional[Dict]:
        """단일 데이터 아이템 처리 및 검증"""
        is_valid, error_msg = self.validate_item(item)
        if not is_valid:
            return None
        
        new_item = item.copy()
        new_item["instruction"] = self.english_instruction
        new_item["category"] = self.extract_categories(item["input"])
        return new_item
    
    def process_data(self, data_list: List[Dict], verbose: bool = True) -> Tuple[List[Dict], List[Dict]]:
        """
        전체 데이터 리스트 처리
        
        Returns:
            (처리된 데이터, 실패한 데이터)
        """
        processed = []
        failed = []
        
        for i, item in enumerate(data_list):
            if verbose:
                print(f"처리 중: {i+1}/{len(data_list)}", end='\r')
            
            result = self.process_single_item(item)
            if result:
                processed.append(result)
            else:
                failed.append(item)
        
        if verbose:
            print(f"✓ {len(processed)}/{len(data_list)} 처리 완료" + " " * 20)
        
        return processed, failed
    
    def save_to_json(self, data: List[Dict], filename: str):
        """JSON 형식으로 저장"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"✓ JSON 저장: {filename}")
    
    def save_to_jsonl(self, data: List[Dict], filename: str):
        """JSONL 형식으로 저장 (Hugging Face 학습 포맷)"""
        with open(filename, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"✓ JSONL 저장: {filename}")
    
    def generate_statistics(self, data: List[Dict]) -> Dict:
        """데이터 통계 생성"""
        categories = [item.get("category", "other") for item in data]
        all_cats = []
        for cat_str in categories:
            all_cats.extend([c.strip() for c in cat_str.split(",")])
        
        return {
            "total_samples": len(data),
            "category_distribution": dict(Counter(all_cats)),
            "avg_input_length": sum(len(item["input"].split()) for item in data) / len(data) if data else 0,
            "avg_output_length": sum(len(item["output"].split()) for item in data) / len(data) if data else 0,
            "min_input_length": min(len(item["input"].split()) for item in data) if data else 0,
            "max_input_length": max(len(item["input"].split()) for item in data) if data else 0,
        }
    
    def generate_report(self, data: List[Dict], filename: str = "processing_report.txt"):
        """처리 결과 리포트 생성"""
        stats = self.generate_statistics(data)
        
        report = f"""
╔══════════════════════════════════════════════════════════════╗
║           HTP 데이터 처리 리포트                              ║
╚══════════════════════════════════════════════════════════════╝

📊 기본 통계:
  - 총 샘플 수: {stats['total_samples']}
  - Input 평균 길이: {stats['avg_input_length']:.1f} 단어
  - Output 평균 길이: {stats['avg_output_length']:.1f} 단어
  - Input 최소/최대 길이: {stats['min_input_length']} / {stats['max_input_length']} 단어

📈 카테고리 분포:
"""
        for cat, count in sorted(stats['category_distribution'].items(), key=lambda x: x[1], reverse=True):
            percentage = (count / stats['total_samples']) * 100 if stats['total_samples'] > 0 else 0
            bar = "█" * int(percentage / 5) + "░" * (20 - int(percentage / 5))
            report += f"  - {cat:15s}: {count:4d} ({percentage:5.1f}%) {bar}\n"
        
        report += f"""
✅ 처리 완료!
   - Instruction 변환: 한국어 → 영어
   - Category 추출: input 텍스트 기반
   - 데이터 검증: 완료

처리 시간: {stats['total_samples']} 샘플
"""
        
        print(report)
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"📄 리포트 저장: {filename}\n")


    

In [17]:
# 설정값 직접 지정 (변경하려면 코드 수정)
input_file = 'htp_output1_eng/htp_alpaca_format.json'
output_dir = './htp_processed_eng'
use_jsonl = True
use_json = False
report_filename = 'processing_report.txt'

# 출력 디렉토리 생성
Path(output_dir).mkdir(parents=True, exist_ok=True)

print("\n" + "=" * 60)
print("📝 HTP 데이터 처리 시작")
print("=" * 60 + "\n")

# 데이터 로드
print(f"📂 데이터 로드 중: {input_file}")
try:
    with open(input_file, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {input_file}")
   
except json.JSONDecodeError:
    print(f"❌ JSON 파일 형식이 잘못되었습니다: {input_file}")

print(f"✓ {len(raw_data)}개 샘플 로드 완료\n")



📝 HTP 데이터 처리 시작

📂 데이터 로드 중: htp_output1_eng/htp_alpaca_format.json
✓ 2257개 샘플 로드 완료



In [18]:
# 프로세서 생성 및 데이터 처리
processor = HTPDataProcessor()
processed_data, failed_data = processor.process_data(raw_data)

if failed_data:
    print(f"\n⚠️ {len(failed_data)}개 샘플 처리 실패\n")

# 통계 생성 및 리포트
report_path = Path(output_dir) / report_filename
processor.generate_report(processed_data, str(report_path))

# 데이터 저장
print("💾 데이터 저장 중...\n")

if use_jsonl or (not use_json):
    processor.save_to_jsonl(
        processed_data,
        str(Path(output_dir) / 'htp_data_processed.jsonl')
    )

if use_json or (not use_jsonl):
    processor.save_to_json(
        processed_data,
        str(Path(output_dir) / 'htp_data_processed.json')
    )

print("\n✨ 모든 처리가 완료되었습니다!\n")


✓ 2257/2257 처리 완료                    

╔══════════════════════════════════════════════════════════════╗
║           HTP 데이터 처리 리포트                              ║
╚══════════════════════════════════════════════════════════════╝

📊 기본 통계:
  - 총 샘플 수: 2257
  - Input 평균 길이: 9.2 단어
  - Output 평균 길이: 41.3 단어
  - Input 최소/최대 길이: 4 / 22 단어

📈 카테고리 분포:
  - house          : 1023 ( 45.3%) █████████░░░░░░░░░░░
  - person         :  894 ( 39.6%) ███████░░░░░░░░░░░░░
  - tree           :  691 ( 30.6%) ██████░░░░░░░░░░░░░░
  - other          :  252 ( 11.2%) ██░░░░░░░░░░░░░░░░░░

✅ 처리 완료!
   - Instruction 변환: 한국어 → 영어
   - Category 추출: input 텍스트 기반
   - 데이터 검증: 완료

처리 시간: 2257 샘플

📄 리포트 저장: htp_processed_eng\processing_report.txt

💾 데이터 저장 중...

✓ JSONL 저장: htp_processed_eng\htp_data_processed.jsonl

✨ 모든 처리가 완료되었습니다!



In [19]:
# 단일 카테고리만 추출
training_data = [item for item in processed_data
                 if item['category'] not in ('other',) and ',' not in item['category']]

# 저장
from pathlib import Path
Path('./htp_processed_eng').mkdir(exist_ok=True)
processor.save_to_jsonl(training_data, './htp_processed_eng/filtered_data.jsonl')

# 확인
print(f"필터링 결과: {len(training_data)}개 (전체 {len(processed_data)}개 중)")

✓ JSONL 저장: ./htp_processed_eng/filtered_data.jsonl
필터링 결과: 1453개 (전체 2257개 중)


In [21]:
from collections import Counter

print("\n필터링된 데이터 통계\n")

# 1. 기본 정보
print(f"총 샘플: {len(training_data)}개")

# 2. 카테고리
categories = Counter(item['category'] for item in training_data)
for cat, count in categories.items():
    print(f"  {cat}: {count}개")

# 3. 길이
input_lens = [len(item['input'].split()) for item in training_data]
output_lens = [len(item['output'].split()) for item in training_data]

print(f"\nInput  길이: 평균 {sum(input_lens)/len(input_lens):.1f} 단어")
print(f"Output 길이: 평균 {sum(output_lens)/len(output_lens):.1f} 단어")

# 4. 품질
empty_in = sum(1 for x in training_data if not x['input'].strip())
empty_out = sum(1 for x in training_data if not x['output'].strip())

print(f"\n데이터 품질: {'✓ 양호' if empty_in == 0 and empty_out == 0 else '⚠ 확인'}")



필터링된 데이터 통계

총 샘플: 1453개
  tree: 408개
  person: 458개
  house: 587개

Input  길이: 평균 8.6 단어
Output 길이: 평균 40.2 단어

데이터 품질: ✓ 양호
